In [1]:
#Boston AirBnB data analysis
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline
pd.options.mode.chained_assignment = None

In [2]:
#start with listings analysis 
listings = pd.read_csv('listings.csv')

In [3]:
listings.shape

(3585, 95)

In [4]:
#change preview settings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
#we are going to work mostly with the price check if price as nulls
listings['price'].isna().sum()

0

In [50]:
listings.head()
#create smaller dataset to work on 
df = listings[['id','host_since','host_response_time','host_response_rate','host_acceptance_rate', 'host_is_superhost', 'host_has_profile_pic',
              'host_identity_verified', 'neighbourhood_cleansed','latitude', 'longitude','property_type','room_type',
              'accommodates','bathrooms','bedrooms','bed_type','amenities','price','guests_included','cleaning_fee',
              'security_deposit','extra_people','minimum_nights','number_of_reviews','review_scores_rating',
              'review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','cancellation_policy',
               'review_scores_communication','review_scores_location','review_scores_value','instant_bookable']]

In [51]:
df.head()

id  host_since  host_response_time host_response_rate host_acceptance_rate host_is_superhost host_has_profile_pic host_identity_verified neighbourhood_cleansed   latitude  longitude property_type        room_type  accommodates  bathrooms  bedrooms  bed_type                                          amenities    price  guests_included cleaning_fee security_deposit extra_people  minimum_nights  number_of_reviews  review_scores_rating  review_scores_accuracy  review_scores_cleanliness  review_scores_checkin cancellation_policy  review_scores_communication  review_scores_location  review_scores_value instant_bookable
0  12147973  2015-04-15                 NaN                NaN                  NaN                 f                    t                      f             Roslindale  42.282619 -71.133068         House  Entire home/apt             4        1.5       2.0  Real Bed  {TV,"Wireless Internet",Kitchen,"Free Parking ...  $250.00                1       $35.00              NaN        $0.00               2                  0                   NaN                     NaN                        NaN                    NaN            moderate                          NaN                     NaN                  NaN                f
1   3075044  2012-06-07      within an hour               100%                 100%                 f                    t                      t             Roslindale  42.286241 -71.134374     Apartment     Private room             2        1.0       1.0  Real Bed  {TV,Internet,"Wireless Internet","Air Conditio...   $65.00                0       $10.00           $95.00        $0.00               2                 36                  94.0                    10.0                        9.0                   10.0            moderate                         10.0                     9.0                  9.0                t
2      6976  2009-05-11  within a few hours               100%                  88%                 t                    t                      t             Roslindale  42.292438 -71.135765     Apartment     Private room             2        1.0       1.0  Real Bed  {TV,"Cable TV","Wireless Internet","Air Condit...   $65.00                1          NaN              NaN       $20.00               3                 41                  98.0                    10.0                        9.0                   10.0            moderate                         10.0                     9.0                 10.0                f
3   1436513  2013-04-21  within a few hours               100%                  50%                 f                    t                      f             Roslindale  42.281106 -71.121021         House     Private room             4        1.0       1.0  Real Bed  {TV,Internet,"Wireless Internet","Air Conditio...   $75.00                2       $50.00          $100.00       $25.00               1                  1                 100.0                    10.0                       10.0                   10.0            moderate                         10.0                    10.0                 10.0                f
4   7651065  2014-05-11      within an hour               100%                 100%                 t                    t                      t             Roslindale  42.284512 -71.136258         House     Private room             2        1.5       1.0  Real Bed  {Internet,"Wireless Internet","Air Conditionin...   $79.00                1       $15.00              NaN        $0.00               2                 29                  99.0                    10.0                       10.0                   10.0            flexible                         10.0                     9.0                 10.0                f

In [52]:
df.groupby(["neighbourhood_cleansed"])["id"].count()

neighbourhood_cleansed
Allston                    260
Back Bay                   302
Bay Village                 24
Beacon Hill                194
Brighton                   185
Charlestown                111
Chinatown                   71
Dorchester                 269
Downtown                   172
East Boston                150
Fenway                     290
Hyde Park                   31
Jamaica Plain              343
Leather District             5
Longwood Medical Area        9
Mattapan                    24
Mission Hill               124
North End                  143
Roslindale                  56
Roxbury                    144
South Boston               174
South Boston Waterfront     83
South End                  326
West End                    49
West Roxbury                46
Name: id, dtype: int64

In [53]:
#converting cat values into numbers 
col_list = ['cancellation_policy', 'host_response_time','neighbourhood_cleansed','host_is_superhost','host_has_profile_pic','host_identity_verified',
            'neighbourhood_cleansed','property_type','room_type', 'bed_type', 'instant_bookable']
            
for col in df:
    if col in col_list:
        df[col +'_numeric'] = pd.factorize( df[col] )[0];


In [54]:
#convert currency columns into number
curr_list = ['price','cleaning_fee','security_deposit','extra_people']
            
for col in df:
    if col in curr_list:
        df[col] = df[col].replace('[\$,]', '', regex=True).astype(float)

In [55]:
#convert % into numbers
pct_list = ['host_response_rate','host_acceptance_rate']
            
for col in df:
    if col in pct_list:
        df[col] = (df[col].replace('[\%,]', '', regex=True).astype(float)) / 100

In [56]:
#replace host_since with number of day from host_since to today
#first convert to date
df[['host_since']] = df[['host_since']].apply(pd.to_datetime)

df['host_since'] = (pd.to_datetime('today') - df['host_since']).dt.days

In [57]:
#find top 10 amenities
# Get the set of unique words.
uniques = []
for row in df['amenities']:
    row = row.lower()
    row = row.replace('{', ' ').replace('"', '').replace('}', '')
    words = row.split(",") 
    for word in words:
        if word not in uniques:
            uniques.append(word)

# Make a list of (count, unique) tuples.
counts = []
final_amenities = []
for unique in uniques:
    count = 0              # Initialize the count to zero.
    for row in df['amenities']:
        row = row.lower()
        row = row.replace('{', ' ').replace('"', '').replace('}', '')
        words = row.split(",")
        for word in words:     # Iterate over the words.
            if word == unique:   # Is this word equal to the current unique?
                count += 1         # If so, increment the count
    counts.append((count, unique))

counts.sort()            # Sorting the list puts the lowest counts first.
counts.reverse()         # Reverse it, putting the highest counts first.

# Print the ten words with the highest counts and create the list with the ones that appear in more than a 1000 places
for i in range(min(21, len(counts))):
    count, word = counts[i]
    final_amenities.append(word)
    print('%s %d' % (word, count))

heating 3382
kitchen 3272
wireless internet 3070
essentials 2994
smoke detector 2908
air conditioning 2786
 tv 2607
dryer 2476
washer 2475
carbon monoxide detector 2442
shampoo 2421
internet 2160
hangers 1985
family/kid friendly 1886
laptop friendly workspace 1834
iron 1828
hair dryer 1811
cable tv 1652
fire extinguisher 1582
24-hour check-in 1246
first aid kit 1063


In [23]:
# add top 21 amenities as a new boolean columns 
final_amenities

['heating',
 'kitchen',
 'wireless internet',
 'essentials',
 'smoke detector',
 'air conditioning',
 ' tv',
 'dryer',
 'washer',
 'carbon monoxide detector',
 'shampoo',
 'internet',
 'hangers',
 'family/kid friendly',
 'laptop friendly workspace',
 'iron',
 'hair dryer',
 'cable tv',
 'fire extinguisher',
 '24-hour check-in']

In [65]:
#add top 21 amenities as 1/0 values to the table

i = 0
for amenities in final_amenities:
    column_to_add = [] 
    for row in df['amenities']:
        if amenities in row.lower(): 
            column_to_add.append(1)
        else:
            column_to_add.append(0)
    col_name = 'Amenities_' +amenities
    df[col_name] = column_to_add   
    i+=1

In [87]:
#we still have a lot of the NaN data in our fields, let's decide if we need to keep some of those based
#on the correlation price
df.isna().sum()

id                                     0
host_since                             0
host_is_superhost                      0
host_has_profile_pic                   0
host_identity_verified                 0
neighbourhood_cleansed                 0
latitude                               0
longitude                              0
property_type                          0
room_type                              0
accommodates                           0
bathrooms                              0
bedrooms                               0
bed_type                               0
amenities                              0
price                                  0
guests_included                        0
extra_people                           0
minimum_nights                         0
number_of_reviews                      0
review_scores_rating                   0
review_scores_accuracy                 0
review_scores_cleanliness              0
review_scores_checkin                  0
cancellation_pol

In [89]:
df[df.columns[1:]].corr()['price'][:].sort_values(ascending=False)

price                                  1.000000
accommodates                           0.593567
bedrooms                               0.546294
bathrooms                              0.335801
Amenities_cable tv                     0.320597
Amenities_ tv                          0.320597
guests_included                        0.310101
Amenities_family/kid friendly          0.284764
Amenities_air conditioning             0.270229
latitude                               0.241166
longitude                              0.207728
review_scores_location                 0.192760
Amenities_washer                       0.176525
Amenities_24-hour check-in             0.162423
Amenities_kitchen                      0.157260
cancellation_policy_numeric            0.150214
Amenities_dryer                        0.147941
review_scores_cleanliness              0.142950
Amenities_iron                         0.137192
Amenities_hair dryer                   0.130118
host_since                             0

In [69]:
#cleaning fee and security deposit have huge corr with price but they are not the factors in setting up the price so 
#I am going to drop those columns. High deposit = high price so this may mess up the prediction. 
df.drop(['cleaning_fee', 'security_deposit'], axis=1, inplace=True)

In [74]:
#droping the columns host_acceptance_rate, host_response_time_numeric, host_response_rate, host_response_time
#the corr is not big enough with for us to drop 471 rows of data. 
df.drop(['host_acceptance_rate', 'host_response_time_numeric','host_response_rate','host_response_time'], axis=1, inplace=True)

In [80]:
#droping rows that have NaN value in columns: bedrooms, bathrooms,property_type
#it is only a couple of rows 
df = df.dropna(subset= ['bedrooms','bathrooms','property_type'],axis = 0)                           

In [86]:
#at this point I think that we have two ways to go, droping the review columns completely or droping the NaN rows.
#replacing those values with avg doesn't make sense. as AirBnB is review based app I will drop the rows without reviews. 

df = df.dropna(subset= ['review_scores_rating','review_scores_accuracy','review_scores_cleanliness',
                       'review_scores_location','review_scores_value'],axis = 0)
#after that we still have 2700 rows of data to predict on. 

(2736, 59)